In [16]:
import torch

a = torch.arange(1*3*15*15).view(1,3,15,15)
# print(a)
# print(a[:, 1:2])
# print(torch.chunk(a, 4, dim=1)[0][:, :3, :])
# # b = torch.ones(3, 4, 4)
print(a[:, :, :, 0:5])
# # c = torch.cat([a, b[:, :3, :]], dim=1)
# # print(c)

tensor([[[[  0,   1,   2,   3,   4],
          [ 15,  16,  17,  18,  19],
          [ 30,  31,  32,  33,  34],
          [ 45,  46,  47,  48,  49],
          [ 60,  61,  62,  63,  64],
          [ 75,  76,  77,  78,  79],
          [ 90,  91,  92,  93,  94],
          [105, 106, 107, 108, 109],
          [120, 121, 122, 123, 124],
          [135, 136, 137, 138, 139],
          [150, 151, 152, 153, 154],
          [165, 166, 167, 168, 169],
          [180, 181, 182, 183, 184],
          [195, 196, 197, 198, 199],
          [210, 211, 212, 213, 214]],

         [[225, 226, 227, 228, 229],
          [240, 241, 242, 243, 244],
          [255, 256, 257, 258, 259],
          [270, 271, 272, 273, 274],
          [285, 286, 287, 288, 289],
          [300, 301, 302, 303, 304],
          [315, 316, 317, 318, 319],
          [330, 331, 332, 333, 334],
          [345, 346, 347, 348, 349],
          [360, 361, 362, 363, 364],
          [375, 376, 377, 378, 379],
          [390, 391, 392, 393, 394],

In [27]:

def ensure_divisibility(numerator, denominator):
    """Ensure that numerator is divisible by the denominator."""
    assert numerator % denominator == 0, '{} is not divisible by {}'.format(
        numerator, denominator)


def divide(numerator, denominator):
    """Ensure that numerator is divisible by the denominator and return
    the division value."""
    # ensure_divisibility(numerator, denominator)
    return numerator // denominator

def split_tensor_along_last_dim(tensor, num_partitions,
                                kernel_size,
                                contiguous_split_chunks=False):
    """Split a tensor along its last dimension.
    Arguments:
        tensor: input tensor.
        num_partitions: number of partitions to split the tensor
        contiguous_split_chunks: If True, make each chunk contiguous
                                 in memory.
    """
    # Get the size and dimension.
    last_dim = tensor.dim() - 1
    last_dim_size = divide(tensor.size()[last_dim], num_partitions)
    print(last_dim_size)
    # Split.
    tensor_list = torch.split(tensor, last_dim_size, dim=last_dim)
    tensor_custom = [torch.cat([tensor_list[i], tensor[:, :, :, ((i+1)*last_dim_size):((i+1)*last_dim_size)+kernel_size-1]], dim=last_dim) for i in range(num_partitions-1)]
    tensor_custom.append(tensor_list[-1])

    # Note: torch.split does not create contiguous tensors by default.
    if contiguous_split_chunks:
        return tuple(chunk.contiguous() for chunk in tensor_list)

    return tensor_custom


print(split_tensor_along_last_dim(a, 4, 2)[0])

3
tensor([[[[  0,   1,   2,   3],
          [ 15,  16,  17,  18],
          [ 30,  31,  32,  33],
          [ 45,  46,  47,  48],
          [ 60,  61,  62,  63],
          [ 75,  76,  77,  78],
          [ 90,  91,  92,  93],
          [105, 106, 107, 108],
          [120, 121, 122, 123],
          [135, 136, 137, 138],
          [150, 151, 152, 153],
          [165, 166, 167, 168],
          [180, 181, 182, 183],
          [195, 196, 197, 198],
          [210, 211, 212, 213]],

         [[225, 226, 227, 228],
          [240, 241, 242, 243],
          [255, 256, 257, 258],
          [270, 271, 272, 273],
          [285, 286, 287, 288],
          [300, 301, 302, 303],
          [315, 316, 317, 318],
          [330, 331, 332, 333],
          [345, 346, 347, 348],
          [360, 361, 362, 363],
          [375, 376, 377, 378],
          [390, 391, 392, 393],
          [405, 406, 407, 408],
          [420, 421, 422, 423],
          [435, 436, 437, 438]],

         [[450, 451, 452, 453],
  